# Домашнее задание №2

**

## Часть 1. ML workflow (**всего 5 баллов**)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы. 

In [15]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [16]:
#your code here
X = df.iloc[:,:-1]
y = df['quality']

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [17]:
#your code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [36]:
#your code here
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred_train = linreg.predict(X_train)

In [37]:
y_pred_test = linreg.predict(X_test)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [38]:
print("Train MSE: {}".format(mean_squared_error(y_train, y_pred_train)))
print("Test MSE: {}".format(mean_squared_error(y_test, y_pred_test)))

Train MSE: 0.40893337932816853
Test MSE: 0.4424926570118656


In [9]:
# На тесте МSE больше, чем на трейне, что в целом нормальный паттерн

In [39]:
print("Train R2: {}".format(r2_score(y_train, y_pred_train)))
print("Test R2: {}".format(r2_score(y_test, y_pred_test)))

Train R2: 0.36834585925746144
Test R2: 0.33096448330839334


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [40]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, random_state=None, shuffle=True)
kf.get_n_splits(X)

5

In [41]:
cv_res = cross_validate(linreg,
                     X,
                     y,
                     scoring='r2', 
                     cv=kf # тут можно было указать просто число фолдов, тогда бы параметры random_state and shuffle были дефолтными
                    )

print(f"r2: {cv_res['test_score']}")
print(f"mean test r2 = {cv_res['test_score'].mean()}")

r2: [0.37138086 0.36464311 0.25783418 0.3406214  0.42268199]
mean test r2 = 0.35143230793732083


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [42]:
from sklearn.linear_model import Lasso

n_alphas = 10
alphas = np.linspace(0.1, 1.1, n_alphas) 

linlasso=Lasso()

In [43]:
#подбор параметра alpha по тренировочным данным (Xtrain, ytrain) по сетке
from sklearn.model_selection import GridSearchCV

params = {'alpha':alphas}

cv = GridSearchCV(linlasso,
                  params,
                  scoring='r2',
                  cv=5
                 )
cv.fit(X, y)

print(cv.best_params_)

{'alpha': 0.1}


In [44]:
for alph in np.arange(0.1, 1.1, 0.1):
    if alph == 0:
        alph += 0.00000001
        
    lin_lasso = Lasso(alpha=alph)
    lin_lasso.fit(X_train, y_train)
    
    
    y_pred_train_lasso = lin_lasso.predict(X_train)
    y_pred_test_lasso = lin_lasso.predict(X_test)
    
    
    print('alpha={}'.format(alph))
    print('Train MSE:', mean_squared_error(y_train, y_pred_train_lasso))
    print('Test MSE:', mean_squared_error(y_test, y_pred_test_lasso))
    print('Train R2:', {r2_score(y_train, y_pred_train_lasso)})                    
    print('Test R2:', {r2_score(y_test, y_pred_test_lasso)})  

alpha=0.1
Train MSE: 0.4976590172927974
Test MSE: 0.4981203821491247
Train R2: {0.2312968448129692}
Test R2: {0.2468570450496227}
alpha=0.2
Train MSE: 0.5317640353683264
Test MSE: 0.5338158906476648
Train R2: {0.1786169292656029}
Test R2: {0.19288651561041892}
alpha=0.30000000000000004
Train MSE: 0.5769456768320785
Test MSE: 0.5766952115768643
Train R2: {0.10882763751590407}
Test R2: {0.12805427901020106}
alpha=0.4
Train MSE: 0.6277282298641573
Test MSE: 0.6251195239372918
Train R2: {0.03038696350467751}
Test R2: {0.05483818304315424}
alpha=0.5
Train MSE: 0.6299965014542217
Test MSE: 0.6267358131825301
Train R2: {0.026883304437895883}
Test R2: {0.05239440257996297}
alpha=0.6
Train MSE: 0.6323842199314446
Test MSE: 0.6286315131091219
Train R2: {0.023195143139976948}
Test R2: {0.04952816161577556}
alpha=0.7000000000000001
Train MSE: 0.6325081708672464
Test MSE: 0.6292640862442844
Train R2: {0.023003683783010986}
Test R2: {0.04857173016403882}
alpha=0.8
Train MSE: 0.6326511911777868
Test 

### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [45]:
print("Наилучший алгоритм: ", cv.best_estimator_)
print("Наилучшее качество: ", cv.best_score_)

Наилучший алгоритм:  Lasso(alpha=0.1)
Наилучшее качество:  0.17886394469115854


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [46]:
lasso_best = Lasso(alpha=cv.best_estimator_.alpha)
lasso_best.fit(X_train, y_train)
y_pred_lb = lasso_best.predict(X_test)
print('Test R2:', {r2_score(y_test, y_pred_lb)})                    

Test R2: {0.2468570450496227}


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [47]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')


pipeline_linreg = Pipeline([('poly', PolynomialFeatures(degree=2)),
                       ('logit', LinearRegression())])


In [48]:
cv_res = cross_validate(pipeline_linreg,
                            X,
                            y,
                            cv=5,
                            scoring='r2'
                           )


In [49]:
print('Mean r2: ', cv_res['test_score'].mean())

Mean r2:  0.23009616560784463


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [50]:
pipeline_linreg.fit(X_train, y_train)

y_pred_train_pipeline = pipeline_linreg.predict(X_train)
y_pred_test_pipeline = pipeline_linreg.predict(X_test)

print('Train MSE: ', mean_squared_error(y_train, y_pred_train_pipeline))
print('Test MSE: ', mean_squared_error(y_test, y_pred_test_pipeline))

print('Train r2: ', r2_score(y_train, y_pred_train_pipeline))
print('Test r2: ', r2_score(y_test, y_pred_test_pipeline))




Train MSE:  0.347565616304894
Test MSE:  0.4576929085442632
Train r2:  0.4631368535398107
Test r2:  0.3079821626378215


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

***Ответ***

1) Исходная модель показала 

Train R2: 0.36834585925746144
Test R2: 0.33096448330839334
Качество модели не прямо хорошее, хотелось бы лучше. При этом можно сказать, что модель обучилась хорошо на трейне, дала не сильно более низкий р-квадрат на тесте, как и мсе. 

Train MSE: 0.40893337932816853
Test MSE: 0.4424926570118656


2) L1-регуляризация не помогла, качество осталось плюс минус на таком же уровне  - если считать по cross validation best estimator с lasso, то р-квадрат также маленький

Test R2: {0.2468570450496227}

3) Частично помогло. На трейне р-квадрат подрос, можно даже сказать значительно, но все еще не устраивает, честно говоря, хотя для реальных данных уже очень даже неплохо. МСЕ упало на трейне, в общем-то можно подумать о том, что мы стали приближаться к истинной форме зависимости, добавив степени, но на тесте р-квадрат стал хуже, чем в самой первой модели, а мсе побольше, так что заподазриваем признаки переобучения

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [18]:
from sklearn.linear_model import Ridge



for a in np.arange(0, 1.1, 0.1):
    if a == 0:
        a += 0.00000001
        
    ridge = Ridge(alpha=a)
    ridge.fit(X_train, y_train)
    
    
    y_pred_train_1 = ridge.predict(X_train)
    y_pred_test_1 = ridge.predict(X_test)
    
    
    print('alpha={}'.format(a))
    print('Train MSE:', mean_squared_error(y_train, y_pred_train_1))
    print('Test MSE:', mean_squared_error(y_test, y_pred_test_1))
    print('Train R2:', {r2_score(y_train, y_pred_train_1)})                    
    print('Test R2:', {r2_score(y_test, y_pred_test_1)})                    


alpha=1e-08
Train MSE: 0.41605816315154626
Test MSE: 0.42264004031848246
Train R2: {0.3572835610761663}
Test R2: {0.3600361116373113}
alpha=0.1
Train MSE: 0.41637290154398776
Test MSE: 0.42255209528495574
Train R2: {0.3567973609322962}
Test R2: {0.36016927849385316}
alpha=0.2
Train MSE: 0.41641485479198365
Test MSE: 0.42269408318942386
Train R2: {0.35673255258447056}
Test R2: {0.35995427962300464}
alpha=0.30000000000000004
Train MSE: 0.4164729778303303
Test MSE: 0.4228384700418117
Train R2: {0.35664276554137386}
Test R2: {0.3597356482519377}
alpha=0.4
Train MSE: 0.41654309276524837
Test MSE: 0.4229823959046734
Train R2: {0.35653445371077663}
Test R2: {0.35951771491376294}
alpha=0.5
Train MSE: 0.41662174617306563
Test MSE: 0.4231239950456819
Train R2: {0.3564129518568083}
Test R2: {0.3593033047106967}
alpha=0.6000000000000001
Train MSE: 0.4167063803790345
Test MSE: 0.4232621159668541
Train R2: {0.356282211012645}
Test R2: {0.3590941612473585}
alpha=0.7000000000000001
Train MSE: 0.416795

In [20]:
ridge_best = Ridge(alpha=1e-08)
ridge_best.fit(X_train, y_train)

y_pred_train_best = ridge_best.predict(X_train)
y_pred_test_best = ridge_best.predict(X_test)
    

print('Train MSE:', mean_squared_error(y_train, y_pred_train_best))
print('Test MSE:', mean_squared_error(y_test, y_pred_test_best))
print('Train R2:', {r2_score(y_train, y_pred_train_best)})                    
print('Test R2:', {r2_score(y_test, y_pred_test_best)})   

Train MSE: 0.41605816315154626
Test MSE: 0.42264004031848246
Train R2: {0.3572835610761663}
Test R2: {0.3600361116373113}


## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [3]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [4]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# create a grid from all combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_df = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_df.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [5]:
all_df


,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0
...,...,...,...,...
10768834,59,22162,33,0.0
10769024,59,22163,33,0.0
10769690,59,22164,33,0.0
10771216,59,22166,33,0.0


### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`. 

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [6]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_df.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_df['item_target_enc'] = all_df['item_id'].map(item_id_target_mean)

# Fill NaNs
all_df['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_df['item_target_enc'].values
print(np.corrcoef(all_df['target'].values, encoded_feature)[0][1])

0.4830386988621791


#### Способ 2

In [7]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_df['item_target_enc'] = all_df.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_df['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_df['item_target_enc'].values
print(np.corrcoef(all_df['target'].values, encoded_feature)[0][1])

0.4830386988621791


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection. 

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях. 

In [8]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle=False)

In [10]:
for train_index, test_index in kf.split(all_df):
    X_train = all_df.iloc[train_index]   #обрежем наши фолды по индексам
    X_test = all_df.iloc[test_index]
    
    item_id_target_mean = X_train.groupby('item_id').target.mean()
 
    
    X_test['item_target_enc'] = X_test['item_id'].map(item_id_target_mean)
  
    
    all_df.iloc[test_index] = X_test #заполним часть данных нашей all_data данными, полученными на одной из валидаций



    
all_df['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = all_df['item_target_enc'].values

corr = np.corrcoef(all_df['target'].values, encoded_feature)[0][1]
print(corr)

<ipython-input-10-a854b5c5d9dd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['item_target_enc'] = X_test['item_id'].map(item_id_target_mean)


0.41645907127988024


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`. 

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [11]:
all_df['n_objects'] = 1 
sums = all_df.groupby('item_id', as_index = False).sum()
all_df['item_target_enc'] = 0.3343
all_df['n_objects'] = all_df.groupby('item_id')['n_objects'].transform('sum')
all_df['target_sum'] = all_df.groupby('item_id')['target'].transform('sum')
all_df['item_target_enc'] = (all_df['target_sum'] - all_df['target'] ) / (all_df['n_objects'] - 1)
encoded_feature = all_df['item_target_enc'].values

corr = np.corrcoef(all_df['target'].values, encoded_feature)[0][1]
print(corr)

0.480384831129305


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [12]:
all_df['n_objects'] = 1
all_df['n_rows'] = all_df.groupby('item_id').n_objects.transform('sum')

all_df['item_target_mean'] = all_df.groupby('item_id').target.transform('mean')

all_df['item_target_enc'] = ((all_df['item_target_mean'] * all_df['n_rows']) + 0.3343 * 100) / (all_df['n_rows'] + 100)
encoded_feature = all_df['item_target_enc'].values

corr = np.corrcoef(all_df['target'].values, encoded_feature)[0][1]
print(corr)

0.4818198797097282


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [13]:
all_df['sum_target'] = all_df.groupby('item_id')['target'].cumsum()
all_df['n_rows'] = all_df.groupby('item_id').cumcount()
all_df['sum_target'] = all_df['sum_target'] - all_df['target']
all_df['item_target_enc'] = all_df['sum_target'] / all_df['n_rows']
all_df['item_target_enc'].fillna(0.3343, inplace=True) 


encoded_feature = all_df['item_target_enc'].values
corr = np.corrcoef(all_df['target'].values, encoded_feature)[0][1]
print(corr)

0.5025245211081701


Ожидаемый ответ 0.5025